In [1]:
import os
#os.environ["WANDB_DISABLED"] = "true"

In [2]:
import logging
log = logging.getLogger()
log.handlers.clear()
log.addHandler(logging.StreamHandler())
log.setLevel(logging.WARNING)

In [3]:
from foresight.datasets.data_collator import CollataAndPad

from foresight.utils import pickle
from foresight.tokenizers.simple_map_tokenizer import SimpleMapTokenizer
# from medcat.cdb import CDB
from foresight.datasets.data_collator import CollataAndPad
from foresight.metrics.next_concept_prediction import precision, metrics_data2df, ComputePrecisionHF
from transformers import GPT2Config, GPT2LMHeadModel, Trainer, TrainingArguments
# from medcat.cat import CAT
# from foresight.models.lucid_transformers import LucidLM2HF
from transformers import SchedulerType

from datasets import Dataset
import math
import datasets
import numpy as np
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import shutil
import random
import pandas as pd
from pathlib import Path

from foresight.tokenizers import PreTrainedTokenizerFastWithPositionIDPadding
from foresight.datasets.data_collator_v2 import DataCollatorForLanguageModelingMaskStaticVariables

In [4]:
from torch.utils.data import DataLoader
import datasets
import time

In [5]:
OUTPUT_DIR = Path.cwd() / "outputs"
SAVE_TOKENIZER_PATH = OUTPUT_DIR / "tokenizer"
SAVE_ENCODED_DATASET_PATH = OUTPUT_DIR / "encoded_dataset"
MODEL_LOGS_DIR = OUTPUT_DIR / "model_logs" / time.strftime("%Y_%m_%d_%H_%M_%S")

NUM_STATIC_VARIABLES = 3

In [6]:
encoded_dataset = datasets.load_from_disk(SAVE_ENCODED_DATASET_PATH)

In [7]:
tokenizer = PreTrainedTokenizerFastWithPositionIDPadding.from_pretrained(SAVE_TOKENIZER_PATH)
data_collator = DataCollatorForLanguageModelingMaskStaticVariables(tokenizer=tokenizer, mlm=False, num_static_variables=NUM_STATIC_VARIABLES)

In [8]:
dataset_train = DataLoader(
    encoded_dataset["train"], batch_size=2, shuffle=False, collate_fn=data_collator
)
next(iter(dataset_train))

{'position_ids': tensor([[ 0,  0,  0,  1,  1,  2,  3,  3,  4,  5,  5,  6,  7,  7,  8,  9,  9, 10,
         11, 11, 12, 13, 13, 14, 15, 15, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  5,
          6,  6,  6,  7,  7,  7,  8,  8,  8,  9,  9,  9, 10, 10, 10, 11, 11, 11,
         12, 12, 12, 13, 13, 13]]), 'input_ids': tensor([[33, 21, 25,  2, 10,  2,  2,  9,  2,  2,  3,  2,  2,  7,  2,  2,  5,  2,
          2,  4,  2,  2,  8,  2,  2,  6,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1],
        [35, 21, 26,  2, 34, 32,  2, 30, 31,  2, 28, 29,  2, 23, 24,  2, 18, 22,
          2, 10, 19,  2,  9, 11,  2,  3, 14,  2,  7, 16,  2,  5, 12,  2,  4, 15,
          2,  8, 17,  2,  6, 13]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0

# Create GPT2

In [9]:
# Make a new model
config = GPT2Config(
    vocab_size=tokenizer.vocab_size + 10,
    # n_positions=16,
    # n_embd=16,
    # n_layer=2,
    # n_head=2,
    bos_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.pad_token_id
)
model = GPT2LMHeadModel(config)

In [10]:
batch = next(iter(dataset_train))

In [11]:
model.transformer.wte.weight.shape

torch.Size([58, 768])

In [12]:
model.transformer.wpe.weight.shape

torch.Size([1024, 768])

In [13]:
model.transformer.wpe.num_embeddings

1024

In [14]:
batch["position_ids"].min()

tensor(0)

# Trainer

In [15]:
# compute_metrics = ComputePrecisionHF(tokenizer.id2tkn, 
#                                      prediction_scope='time_range', 
#                                      topk=1, 
#                                      start=0, 
#                                      return_all_metrics=False, 
#                                      batch_size=1000, 
#                                      select_token_types=all_types,
#                                      type_data=test_set_to_use['token_type'],
#                                      token_type2tokens=tokenizer.token_type2tokens,
#                                      time_data=test_set_to_use['time'], 
#                                      time_range=30*24*60*60,
#                                      ignore_label_status=False,
#                                      min_time_left=24*60*60)

In [16]:
MODEL_LOGS_DIR.mkdir(parents=True, exist_ok=True)

training_args = TrainingArguments(
    output_dir=MODEL_LOGS_DIR,          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    weight_decay=1e-2,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    warmup_ratio=0.01,
    learning_rate= 3.14e-04,
    eval_accumulation_steps=1,
    gradient_accumulation_steps=16,
    do_eval=True,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    # metric_for_best_model='eval_loss',
    
    load_best_model_at_end=True,
    lr_scheduler_type=SchedulerType.LINEAR,
    # use_cpu=True
)

In [17]:
# import wandb

In [18]:
# wandb.init(project='timecat', entity='wish', name=RUN_NAME + '-gpt-16-16_1day_no_base_data')

In [19]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=encoded_dataset['train'],         # training dataset
    eval_dataset=encoded_dataset['test'],             # evaluation dataset
    # compute_metrics=compute_metrics,
    data_collator=data_collator,
    # prediction_loss_only=True
    # tokenizer=None,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,No log,9.654330
1,No log,2.690882
2,No log,2.631073
3,No log,2.623425
4,No log,2.437474
5,No log,2.424219
6,No log,2.383392
8,No log,2.375720
9,No log,2.376614


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=30, training_loss=3.148661804199219, metrics={'train_runtime': 37.5288, 'train_samples_per_second': 213.17, 'train_steps_per_second': 0.799, 'total_flos': 188767162368000.0, 'train_loss': 3.148661804199219, 'epoch': 9.6})

In [21]:
trainer.save_model(MODEL_PATH)

NameError: name 'MODEL_PATH' is not defined

# Test

In [ ]:
all_types = set(['T-11', 'T-45', 'T-55', 'T-18', 'T-26', 'T-40', 'T-39', 'T-49', 'T-29', 'T-34', 
                 'T-9', 'T-33', 'T-44', 'T-6', 'T-27', 'T-38', 'T-35', 'T-3', 'T-58'])

In [ ]:
test_set_to_use = encoded_dataset['test']
test_set_to_use

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=None,         # training dataset
    eval_dataset=None,             # evaluation dataset
    compute_metrics=None,
    data_collator=collate_fn,
    tokenizer=None,
)

In [ ]:
def get_metrics(metrics_data=None, test_set_to_use=None, trainer=None, m_file=None, f_name=None):
    size = 1000
    for i in range(int(math.ceil(len(test_set_to_use) / size))):
        _dataset = Dataset.from_dict(test_set_to_use[i*size:(i+1)*size])
        compute_metrics.time_data = _dataset['time']
        compute_metrics.type_data = _dataset['token_type']
        if len(_dataset):
            p = trainer.predict(_dataset)
            metrics_data = compute_metrics(p, metrics_data)['metrics_data']
    m_file.write("{}, {}, {}, {}\n".format(f_name, metrics_data['precision']['all'], 
                                 metrics_data['precision']['new'], 
                                 metrics_data['precision']['old'],
                                 metrics_data['recall']['all'],
                                 metrics_data['recall']['new'],
                                 metrics_data['recall']['old']))
    print(f_name,
          metrics_data['precision']['all'], 
          metrics_data['precision']['new'], 
          metrics_data['precision']['old'],
          metrics_data['recall']['all'],
          metrics_data['recall']['new'],
          metrics_data['recall']['old']) 
    pickle.dump(metrics_data, f_name)

    return metrics_data

In [ ]:
m_file = open("./metrics/summary.txt", 'w', buffering=1)
m_file.write("file_name, precision all, precision new, precision old\n")

for types in [all_types, {'T-11'}, {'T-55'}, {'T-18'}, {'T-39'}]:
    _types = list(types)[0] if len(types) == 1 else 'all_types'
    for timerange in [30, 365, 1000000]:
        compute_metrics = ComputePrecisionHF(tokenizer.id2tkn, 
                                         prediction_scope='time_range', 
                                         topk=1, # 1, 5, 10
                                         start=0, # 0, 10, 20, 50, 100
                                         return_all_metrics=True, 
                                         batch_size=1000, 
                                         select_token_types=types,
                                         type_data=test_set_to_use['token_type'],
                                         token_type2tokens=tokenizer.token_type2tokens,
                                         time_data=test_set_to_use['time'], 
                                         time_range=timerange*24*60*60, #30, 365, 1000000
                                         ignore_label_status=False,
                                         min_time_left=24*60*60)
        f_name = f"./metrics/start-0_topk-1_time_range-{timerange}_types-{_types}.pickle"
        get_metrics(None, test_set_to_use, trainer, m_file, f_name)

    for topk in [5, 10]:
        compute_metrics = ComputePrecisionHF(tokenizer.id2tkn, 
                                         prediction_scope='time_range', 
                                         topk=topk, # 1, 5, 10
                                         start=0, # 0, 10, 20, 50, 100
                                         return_all_metrics=True, 
                                         batch_size=1000, 
                                         select_token_types=types,
                                         type_data=test_set_to_use['token_type'],
                                         token_type2tokens=tokenizer.token_type2tokens,
                                         time_data=test_set_to_use['time'], 
                                         time_range=30*24*60*60, #30, 365, 1000000
                                         ignore_label_status=False,
                                         min_time_left=24*60*60)
        f_name = f"./metrics/start-0_topk-{topk}_time_range-30_types-{_types}.pickle"
        get_metrics(None, test_set_to_use, trainer, m_file, f_name)
m_file.close()

# Test Death

In [ ]:
all_types = set(['death'])

In [ ]:
test_set_to_use

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=None,         # training dataset
    eval_dataset=None,             # evaluation dataset
    compute_metrics=None,
    data_collator=collate_fn,
    tokenizer=None,
)

In [ ]:
def get_metrics(metrics_data=None, test_set_to_use=None, trainer=None, m_file=None, f_name=None):
    size = 1000
    for i in range(int(math.ceil(len(test_set_to_use) / size))):
        _dataset = Dataset.from_dict(test_set_to_use[i*size:(i+1)*size])
        compute_metrics.time_data = _dataset['time']
        compute_metrics.type_data = _dataset['token_type']
        if len(_dataset):
            p = trainer.predict(_dataset)
            metrics_data = compute_metrics(p, metrics_data)['metrics_data']
    m_file.write("{}, {}, {}, {}\n".format(f_name, metrics_data['precision']['all'], 
                                 metrics_data['precision']['new'], 
                                 metrics_data['precision']['old']))
    print(f_name,
          metrics_data['precision']['all'], 
          metrics_data['precision']['new'], 
          metrics_data['precision']['old'])
    pickle.dump(metrics_data, f_name)

In [ ]:
compute_metrics = ComputePrecisionHF(tokenizer.id2tkn, 
                                 topk=1, # 1, 5, 10
                                 start=0, # 0, 10, 20, 50, 100
                                 return_all_metrics=True, 
                                 batch_size=1000, 
                                 type_data=test_set_to_use['token_type'],
                                 token_type2tokens=tokenizer.token_type2tokens,
                                 time_data=test_set_to_use['time'], 
                                 time_range=24*60*60, #30, 365, 1000000
                                 ignore_label_status=False,
                                 min_time_left=0,
                                 concept_id=270)

In [ ]:
metrics_data = None
_dataset = Dataset.from_dict(test_set_to_use[0:1000])
compute_metrics.time_data = _dataset['time']
compute_metrics.type_data = _dataset['token_type']
if len(_dataset):
    p = trainer.predict(_dataset)
    metrics_data = compute_metrics(p, metrics_data)['metrics_data']

In [ ]:
metrics_data

In [ ]:
tokenizer.tkn2id['The patient has died']

In [ ]:
for i in range(len(_dataset['input_ids'])):
    if 270 in _dataset['input_ids'][i]:
        print(i)

In [ ]:
metrics_data = None
size = 1000
for i in range(int(math.ceil(len(test_set_to_use) / size))):
    _dataset = Dataset.from_dict(test_set_to_use[i*size:(i+1)*size])
    compute_metrics.time_data = _dataset['time']
    compute_metrics.type_data = _dataset['token_type']
    if len(_dataset):
        p = trainer.predict(_dataset)
        metrics_data = compute_metrics(p, metrics_data)['metrics_data']

# Hyperparameter search

In [ ]:
from ray.tune.schedulers import PopulationBasedTraining
from ray import tune
from ray.tune import CLIReporter 
import ray

In [ ]:
compute_metrics = ComputePrecisionHF(id2tkn, id2type, prediction_scope='age', topk=1, start=0, batch_size=2000)

In [ ]:
NUM_TRIALS = 20
N_GPU_PER_TRIAL = 1
METRIC_TO_OPTIMIZE = 'eval_precision'

In [ ]:
def get_model(params):
    torch.cuda.empty_cache()
    if params is None:
        params = {}
    
    config = GPT2Config(
        vocab_size=len(embeddings),
        n_positions=MAX_SEQ_LEN+1,
        n_ctx=MAX_SEQ_LEN+1,
        n_embd=params.get('n_embd', 300),
        n_layer=params.get('n_layer', 1),
        n_head=params.get('n_head', 1),
        bos_token_id=tkn2id['<PAD>'],
        eos_token_id=tkn2id['<PAD>']
    )
    model = GPT2LMHeadModel(config)
    
    if params.get('load_weights', 0):
        model.transformer.wte.load_state_dict({'weight': torch.tensor(embeddings, dtype=torch.float32)})
        model.transformer.wte.weight.requires_grad = True
    
    return model

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=128,   # batch size for evaluation
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,
    eval_steps=200,
    learning_rate= 5e-5,
    eval_accumulation_steps=1,
    do_eval=True,
    evaluation_strategy='steps',
    skip_memory_metrics=True,
)

In [ ]:
training_args.n_head = 1
training_args.n_layer = 1
training_args.n_embd = 300
training_args.load_weights = 0

In [ ]:
tune_dataset = encoded_dataset['train'].train_test_split(test_size=0.1)

In [ ]:
tune_train_dataset = tune_dataset['train']
tune_test_dataset = tune_dataset['test']

In [ ]:
trainer = Trainer(
#    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tune_train_dataset,         # training dataset
    eval_dataset=tune_test_dataset,             # evaluation dataset
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    tokenizer=None,
    model_init=get_model,
)

In [ ]:
tune_config = {
    "num_train_epochs": tune.choice([5]),
    "n_head": tune.choice([2, 4, 6]),
}
scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    metric=METRIC_TO_OPTIMIZE,
    mode="max",
    perturbation_interval=1,
    hyperparam_mutations={
        "weight_decay": tune.uniform(0.0, 0.3),
        "learning_rate": tune.uniform(1e-5, 5e-5),
        "per_device_train_batch_size": [16, 32, 64, 128],
        "n_layer": tune.choice([2, 4, 6, 8]),
#       "n_embd": tune.choice([256, 512]),
        "load_weights": tune.choice([0, 1]),
        "warmup_steps": tune.choice([20, 40, 60, 100]),
    })

In [ ]:
import copy
def compute_objective(metrics):
    metrics = copy.deepcopy(metrics)
    eval_precision = metrics.pop('eval_precision')
    
    return eval_precision

In [ ]:
best_model = trainer.hyperparameter_search(
    hp_space=lambda _: tune_config,
    backend="ray",
    n_trials=NUM_TRIALS,
    direction='maximize',
    compute_objective=compute_objective,
    resources_per_trial={
        "cpu": 1,
        "gpu": N_GPU_PER_TRIAL
    },
    scheduler=scheduler,
    keep_checkpoints_num=1,
    checkpoint_score_attr=METRIC_TO_OPTIMIZE,
    stop=None,
    local_dir=RESULTS_HYPERPARAM,
    name="21_May_2021",
    log_to_file=False,
    loggers=None,# (WandbLogger, ),
    )

In [ ]:
best_model

# Saliency 

In [ ]:
import ecco

In [ ]:
lm = ecco.LM(trainer.model, tokenizer, model_name='gpt2')

In [ ]:
ind = 49
print("~~".join([tokenizer.id2tkn[id] for id in encoded_dataset['test'][ind]['input_ids']]))
text = "~~".join([tokenizer.id2tkn[id] for id in encoded_dataset['test'][ind]['input_ids'][1:-1]])

In [ ]:
output = lm.generate(text, generate=10, do_sample=True, temperature=1)

In [ ]:
output.saliency(style="detailed")

# Probability prediction

In [ ]:
from foresight.sight import Sight

In [ ]:
_ = model.eval()

In [ ]:
sight = Sight(tokenizer=tokenizer, device='cuda', model=model)

In [ ]:
cdb.name2cuis['muscle~pain']

In [ ]:
cdb.get_name('pain')

In [ ]:
text = '<ETHNICITY>~~White~~<SEX>~~Male~~<AGE>~~23~~49727002~~386661006'.split("~~")

In [ ]:
# Small with WD
r = sight.next_concepts(text, type_ids=['T-11'], n=40, p_new=True, create_position_ids=False)
print([cdb.get_name(x) for x in text])
for x in r:
    print(x[0], x[1], cdb.get_name(x[0]))